# Object-Detection using [KerasCV](https://github.com/keras-team/keras-cv) and [Weights & Biases](https://wandb.ai/site)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/soumik12345/wandb-addons/blob/main/docs/keras/examples/object_detection_inference.ipynb)

This notebook demonstrates how to use a pre-trained object-detection model from KerasCV and visualize the results using Weights & Biases.

In [ ]:
!pip install --upgrade -q git+https://github.com/keras-team/keras-cv
!pip install --upgrade -q git+https://github.com/soumik12345/wandb-addons
!pip install -q namex
!apt install python3.10-venv
!git clone https://github.com/keras-team/keras-core.git && cd keras-core
!python pip_build.py --install

## Import the Libraries

In [ ]:
import keras_cv
import keras as keras
from keras import ops

import numpy as np

import wandb
from wandb_addons.keras.detection import log_predictions_to_wandb

## Initialize a Weights & Biases run and Set up the Configs

In [ ]:
wandb.init(project="keras-community-days", job_type="detection/inference")

config = wandb.config
config.model_name = "retinanet_resnet50_pascalvoc"
config.image_size = 640

class_ids = [
    "Aeroplane",
    "Bicycle",
    "Bird",
    "Boat",
    "Bottle",
    "Bus",
    "Car",
    "Cat",
    "Chair",
    "Cow",
    "Dining Table",
    "Dog",
    "Horse",
    "Motorbike",
    "Person",
    "Potted Plant",
    "Sheep",
    "Sofa",
    "Train",
    "Tvmonitor",
    "Total",
]
config.class_mapping = dict(zip(range(len(class_ids)), class_ids))

## Fetch the Images

In [ ]:
filepath_1 = keras.utils.get_file(
    origin="https://i.imgur.com/gCNcJJI.jpg"
)
filepath_2 = keras.utils.get_file(
    origin="https://i.imgur.com/M8LR4fz.png"
)
filepath_3 = keras.utils.get_file(
    origin="https://i.imgur.com/H2d3VJC.jpeg"
)
filepath_4 = keras.utils.get_file(
    origin="https://i.imgur.com/eTxzk46.jpeg"
)

image_1 = keras.utils.load_img(filepath_1)
image_1 = np.array(image_1)
image_2 = keras.utils.load_img(filepath_2)
image_2 = np.array(image_2)
image_3 = keras.utils.load_img(filepath_3)
image_3 = np.array(image_3)
image_4 = keras.utils.load_img(filepath_4)
image_4 = np.array(image_4)

## Preprocess the Images

In [ ]:
inference_resizing = keras_cv.layers.Resizing(
    config.image_size,
    config.image_size,
    pad_to_aspect_ratio=True,
    bounding_box_format="xywh"
)
image_1_resized = inference_resizing([image_1])
image_2_resized = inference_resizing([image_2])
image_3_resized = inference_resizing([image_3])
image_4_resized = inference_resizing([image_4])
image_batch = ops.concatenate(
    [
        image_1_resized,
        image_2_resized,
        image_3_resized,
        image_4_resized
    ],
    axis=0
)
image_batch.shape

## Define a pre-trained model from KerasCV

In [ ]:
pretrained_model = keras_cv.models.RetinaNet.from_preset(
    config.model_name, bounding_box_format="xywh"
)

## Perform Inference

In [ ]:
y_pred = pretrained_model.predict(image_batch)
log_predictions_to_wandb(image_batch, y_pred, config.class_mapping)

Need more control over the visualization?
Reduce the threshold for Confidence by uncommenting the following code!

In [ ]:
# prediction_decoder = keras_cv.layers.MultiClassNonMaxSuppression(
#     bounding_box_format="xywh",
#     from_logits=True,
#     iou_threshold=0.01,
#     confidence_threshold=0.01,
# )
# pretrained_model.prediction_decoder = prediction_decoder
# y_pred = pretrained_model.predict(image_batch)
# log_predictions_to_wandb(image_batch, y_pred, config.class_mapping)

Finish the run!

In [ ]:
wandb.finish()